# Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools which are wrapped in PyMC3 by `model.profile`. This function returns a `ProfileStats` object conveying information about the underlying Theano operations. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [9]:
import pandas as pd
from pymc3 import *
from pymc3.math import exp
from pymc3.distributions.timeseries import *

returns = pd.read_csv(get_data('SP500.csv'),index_col=0)

n = returns.shape[0]

with Model() as model:
    sigma = Exponential('sigma', 1. / .02, testval=.1)
    nu = Exponential('nu', 1. / 10)
    s = GaussianRandomWalk('s', sigma ** -2, shape=n)
    r = StudentT('r', nu, lam=exp(-2 * s), observed=returns)
    


Then we call the `profile` function and summarize its return values.

In [10]:
model.profile(model.logpt).summary()


Function profiling
  Message: /Users/ckrapu/miniconda3/envs/pymc3-dev/lib/python3.7/site-packages/pymc3-3.5-py3.7.egg/pymc3/model.py:910
  Time in 1000 calls to Function.__call__: 4.956158e+00s
  Time in Function.fn.__call__: 4.874094e+00s (98.344%)
  Time in thunks: 4.822645e+00s (97.306%)
  Total compile time: 9.606049e-01s
    Number of Apply nodes: 26
    Theano Optimizer time: 4.315219e-01s
       Theano validate time: 3.566027e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 4.510198e-01s
       Import time 0.000000e+00s
       Node make_thunk time 4.486918e-01s
           Node InplaceDimShuffle{x}(sigma) time 4.140620e-01s
           Node Elemwise{Composite{(Switch(Cast{int8}(GE(i0, i1)), (i2 - (i3 * i0)), i4) + i5)}}(sigma, TensorConstant{0}, TensorConstant{3.9120230674743652}, TensorConstant{50.0}, TensorConstant{-inf}, sigma_log__) time 2.099991e-03s
           Node Elemwise{exp,no_inplace}(nu_log__) time 2.029181e-03s
           Node Elemwise{Composi

In [11]:
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /Users/ckrapu/miniconda3/envs/pymc3-dev/lib/python3.7/site-packages/pymc3-3.5-py3.7.egg/pymc3/model.py:910
  Time in 1000 calls to Function.__call__: 6.775435e+00s
  Time in Function.fn.__call__: 6.688420e+00s (98.716%)
  Time in thunks: 6.578395e+00s (97.092%)
  Total compile time: 1.072301e+00s
    Number of Apply nodes: 49
    Theano Optimizer time: 9.815018e-01s
       Theano validate time: 1.133823e-02s
    Theano Linker time (includes C, CUDA code generation/compiling): 5.134106e-02s
       Import time 0.000000e+00s
       Node make_thunk time 4.863691e-02s
           Node Elemwise{Composite{(Switch(Cast{int8}(GE(i0, i1)), (i2 * i0), i1) + i3 + (i4 * i5 * psi((i4 * (i6 + i0))) * i0) + (i7 * i8 * i9) + (i10 * i4 * i11 * psi((i4 * i0)) * i0) + (i4 * i12 * i0) + i13)}}[(0, 0)](nu, TensorConstant{0}, TensorConstant{-0.1}, TensorConstant{1.0}, TensorConstant{0.5}, Sum{acc_dtype=float64}.0, TensorConstant{1.0}, TensorConstant{3.141592653589793}, TensorCons